In [ ]:
import requests
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

# CKAN API base URL
base_url = "https://data.hii.or.th/sv/api/3/action/datastore_search"

# Resource ID for the dataset (replace with your actual resource ID)
resource_id = "e45c665d-8389-4bf1-889b-37c42ca200ee"

# Query parameters (you can adjust this)
params = {
    'resource_id': resource_id,
    'limit': 5000  # Set a limit to the number of records to fetch
}

# Fetch data from the CKAN API
response = requests.get(base_url, params=params)

# Check if the request was successful
if response.status_code == 200:
    # Parse the JSON data into a pandas DataFrame
    records = response.json()['result']['records']
    df = pd.DataFrame(records)
    
    # Display the first few rows of the DataFrame
    print(df.head())
else:
    print(f"Failed to fetch data. Status code: {response.status_code}")

# Example: Dataset preprocessing for AI training
# Assuming we are working with the columns: Rainfall(mm), Province_ID, Initial_Year, Initial_Month

# Select relevant features (you can change these as per your data)
X = df[['Initial_Year', 'Initial_Month', 'Province_ID']]  # Features
y = df['Rainfall(mm)']  # Target

# Train-test split (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Feature scaling (Normalize data)
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Build a simple neural network model using Keras
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)))
model.add(Dense(64, activation='relu'))
model.add(Dense(1))  # Output layer for regression (predicting rainfall)

# Compile the model
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='mean_squared_error')

# Train the model
model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_split=0.2)

# Evaluate the model
loss = model.evaluate(X_test_scaled, y_test)
print(f'Model Loss on Test Data: {loss}')

# Save the model for future use
model.save("rainfall_prediction_model.h5")


In [4]:
import requests
import pandas as pd

# CKAN API base URL (without specifying a resource ID)
base_url = "https://data.hii.or.th/sv/api/3/action/datastore_search"

# Define pagination limit and offset
limit = 5000  # Number of records to fetch in each request
offset = 0  # Start fetching from the first record
all_records = []  # To store all fetched records

while True:
    # Request parameters with pagination
    params = {
        'limit': limit,
        'offset': offset
    }
    
    # Fetch data from CKAN API
    response = requests.get(base_url, params=params)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Parse the JSON data into a dictionary
        data = response.json().get('result', {}).get('records', [])
        
        # If no more records are returned, break the loop
        if not data:
            break
        
        # Append the fetched records to all_records list
        all_records.extend(data)
        
        # Increment the offset for the next request
        offset += limit
    else:
        print(f"Failed to fetch data. Status code: {response.status_code}")
        break

# Convert the data to a pandas DataFrame
df = pd.DataFrame(all_records)

# Check the total number of records fetched
print(f"Total records fetched: {len(df)}")

# Optionally save the data to a CSV file
df.to_csv('all_ckan_data.csv', index=False)

# Display the first few rows of the dataset
print(df.head())


Failed to fetch data. Status code: 409
Total records fetched: 0
Empty DataFrame
Columns: []
Index: []
